In [1]:
# Imports
from train import build_vocab_embedding, build_model
from argparse import ArgumentParser


/home/fanmin/mambaforge/envs/acts_gpu_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# init glove
parser = ArgumentParser()
parser.add_argument(
        "--encoder",
        default="avg",
        choices=["avg", "lstm", "bi-lstm", "bi-lstm-max-pool"],
        type=str)
parser.add_argument("--checkpt", type=str)
glove_vocab, pretrained_embeddings, _, _, _ = build_vocab_embedding()

unique token len 37211


In [10]:
# Create models
CHECK_PT = '/home/fanmin/atcs-lstm/models/avg-epoch=19-val_loss=0.66-val_accuracy=0.72.ckpt' # replace me
args = parser.parse_args(['--encoder', 'avg', '--checkpt', CHECK_PT])
model_avg = build_model(args, pretrained_embeddings)

CHECK_PT = '/home/fanmin/atcs-lstm/models/lstm-epoch=4-val_loss=0.50-val_accuracy=0.80.ckpt' # replace me
args = parser.parse_args(['--encoder', 'lstm', '--checkpt', CHECK_PT])
model_lstm = build_model(args, pretrained_embeddings)

CHECK_PT = '/home/fanmin/atcs-lstm/models/bi-lstm-epoch=4-val_loss=0.50-val_accuracy=0.80.ckpt' # replace me
args = parser.parse_args(['--encoder', 'bi-lstm', '--checkpt', CHECK_PT])
model_bilstm = build_model(args, pretrained_embeddings)

CHECK_PT = '/home/fanmin/atcs-lstm/models/bi-lstm-max-pool-epoch=8-val_loss=0.42-val_accuracy=0.84.ckpt' # replace me
args = parser.parse_args(['--encoder', 'bi-lstm-max-pool', '--checkpt', CHECK_PT])
model_bilstm_max = build_model(args, pretrained_embeddings)

In [11]:
from nltk.tokenize import word_tokenize
import torch
def eval(model, p, h):
    p_ind = glove_vocab(word_tokenize(p))
    h_tk = glove_vocab(word_tokenize(h))
    p_text = torch.tensor([len(p_ind)])
    h_text = torch.tensor([len(h_tk)])
    prep = model(torch.tensor([p_ind]), p_text, torch.tensor([p_ind]), h_text)
    print(f'{p},{h} has predict of {prep}')

In [12]:
# Evluate
p = "two men sitting in the sun ."
h = "nobody is sitting in the shade ."
eval(model_avg, p, h)
eval(model_lstm, p, h)
eval(model_bilstm, p, h)
eval(model_bilstm_max, p, h)


p2 = "a man is walking a dog ."
h2 = "no cat is outside ."
eval(model_avg, p2, h2)
eval(model_lstm, p2, h2)
eval(model_bilstm, p2, h2)
eval(model_bilstm_max, p2, h2)

two men sitting in the sun .,nobody is sitting in the shade . has predict of tensor([[ 0.4113, -0.7574,  0.3910]], grad_fn=<AddmmBackward0>)
two men sitting in the sun .,nobody is sitting in the shade . has predict of tensor([[ 2.4773, -1.0979, -1.4170]], grad_fn=<AddmmBackward0>)
two men sitting in the sun .,nobody is sitting in the shade . has predict of tensor([[ 3.2840, -0.8510, -2.2894]], grad_fn=<AddmmBackward0>)
two men sitting in the sun .,nobody is sitting in the shade . has predict of tensor([[ 2.6835, -0.9080, -1.7141]], grad_fn=<AddmmBackward0>)
a man is walking a dog .,no cat is outside . has predict of tensor([[ 6.4695, -2.2916, -3.9968]], grad_fn=<AddmmBackward0>)
a man is walking a dog .,no cat is outside . has predict of tensor([[ 2.1044,  0.0434, -2.1469]], grad_fn=<AddmmBackward0>)
a man is walking a dog .,no cat is outside . has predict of tensor([[ 1.7867,  0.4956, -2.2483]], grad_fn=<AddmmBackward0>)
a man is walking a dog .,no cat is outside . has predict of tens

By looking a the result, the predicted label for all the models on p and p2 are first label. 
The NLI labels are contradiction, neutral and entailment. Hence all trained models predict contradiction.